# Delta Lake Operations

**Cel szkoleniowy:** Zrozumienie fundamentów Delta Lake i praktyczne zastosowanie operacji CRUD, Time Travel, optymalizacji i Change Data Feed

**Zakres tematyczny:**
- Delta Lake core features: ACID, Delta Log, Schema enforcement
- Schema evolution (additive, automatic)
- Time Travel i Copy-on-write
- CRUD operations: CREATE TABLE, INSERT, UPDATE, DELETE, MERGE INTO
- Optymalizacja: OPTIMIZE, ZORDER BY, VACUUM
- Change Data Feed

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Lakehouse & Delta Lake
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 16.4 LTS lub 17.3 LTS (Spark 4.0)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z 2-4 workers (lub Serverless Compute)

## Wstęp teoretyczny

**Cel sekcji:** Wprowadzenie do Delta Lake jako transakcyjnej warstwy storage nad Data Lake

**Podstawowe pojęcia:**
- **Delta Lake**: Open-source storage layer zapewniający ACID transactions dla Apache Spark
- **Delta Log**: Transakcyjny log przechowujący metadane o wszystkich zmianach w tabeli
- **Schema Enforcement**: Automatyczna walidacja zgodności schematów przy zapisie
- **Time Travel**: Możliwość dostępu do poprzednich wersji danych

**Dlaczego to ważne?**
Delta Lake rozwiązuje fundamentalne problemy Data Lake: brak transakcji, schema drift, trudności z aktualizacjami i quality assurance. Zapewnia niezawodność Data Warehouse z elastycznością Data Lake.

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime, timedelta

# Wyświetl kontekst użytkownika
display(
    spark.createDataFrame([
        (CATALOG, BRONZE_SCHEMA, SILVER_SCHEMA, GOLD_SCHEMA)
    ], ['catalog', 'bronze_schema', 'silver_schema', 'gold_schema'])
)

# Ustaw katalog i schemat jako domyślne
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

## Sekcja 1: Delta Lake Core Features

**Wprowadzenie teoretyczne:**

Delta Lake to warstwa transakcyjna nad Parquet, która zapewnia ACID properties (Atomicity, Consistency, Isolation, Durability). Każda operacja na tabeli Delta jest rejestrowana w Delta Log - JSON pliku zawierającym metadane o zmianach.

**Kluczowe pojęcia:**
- **ACID Transactions**: Wszystkie operacje są atomowe i spójne
- **Delta Log**: `_delta_log/` folder z JSON plikami opisującymi każdą transakcję
- **Schema Enforcement**: Automatyczna walidacja zgodności schematów
- **Unified Batch and Streaming**: Jedna tabela obsługuje zarówno batch jak i streaming

**Zastosowanie praktyczne:**
- Transakcyjne aktualizacje w Data Lake
- Zapewnienie jakości danych poprzez schema validation
- Jednolity dostęp do danych dla batch i streaming workloads

### Przykład 1.1: Utworzenie pierwszej tabeli Delta

**Cel:** Demonstracja tworzenia tabeli Delta i podstawowych właściwości

**Podejście:**
1. Wczytanie danych z Unity Catalog Volume
2. Utworzenie managed table w formacie Delta
3. Eksploracja Delta Log i metadanych

In [0]:
# Wczytaj dane klientów z Unity Catalog Volume
customers_df = (spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(f"{DATASET_BASE_PATH}/customers/customers.csv")
)

**Utwórz managed Delta table:**

In [0]:
# Utwórz managed Delta table
customers_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta")

**Wyświetl wynik:**

In [0]:
display(spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta").limit(5))

**Wyjaśnienie:**

Utworzono managed Delta table w Unity Catalog. Format Delta automatycznie:
- Stworzył `_delta_log/` folder z metadanymi transakcji
- Zarejestrował schemat tabeli w Unity Catalog
- Zastosował kompresję Parquet z dodatkowymi Delta features

### Przykład 1.2: Schema Enforcement w akcji

**Cel:** Demonstracja automatycznej walidacji schematów przy wstawianiu danych

In [0]:
# Sprawdź aktualny schemat tabeli
spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta").printSchema()

In [0]:
# Próba wstawienia danych z niepoprawnym schematem (brakuje kolumn)
invalid_data = spark.createDataFrame([
    ("CUST999999", "Test", "Customer", "invalid_email", "+48 123 456 789")  # Brakuje city, state, country, registration_date, customer_segment
], ["customer_id", "first_name", "last_name", "email", "phone"])

try:
    invalid_data.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta")
except Exception as e:
    display(
        spark.createDataFrame([
            ("Schema enforcement w działaniu", str(e)[:200] + "...")
        ], ["message", "error"])
    )

**Wyjaśnienie:**

Schema enforcement automatycznie odrzucił dane z niepoprawnym typem. Delta Lake porównuje schemat nowych danych ze schematem tabeli i blokuje niezgodne wstawienia, zapewniając consistency.

In [0]:
# Utwórz tabelę z Identity Column i Generated Column
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {CATALOG}.{BRONZE_SCHEMA}.orders_modern (
    order_sk BIGINT GENERATED ALWAYS AS IDENTITY,  -- Surrogate Key
    order_id STRING,
    total_amount DOUBLE,
    order_timestamp TIMESTAMP,
    order_date DATE GENERATED ALWAYS AS (CAST(order_timestamp AS DATE)) -- Auto-calculated
) USING DELTA
""")


In [0]:
display(spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.orders_modern"))


Sprawdźmy wynik. Zwróć uwagę na automatycznie wypełnione kolumny.


In [0]:
# Wstaw dane bez podawania kolumn generowanych
spark.sql(f"""
INSERT INTO {CATALOG}.{BRONZE_SCHEMA}.orders_modern (order_id, total_amount, order_timestamp)
VALUES 
    ('ORD-001', 150.50, current_timestamp()),
    ('ORD-002', 200.00, current_timestamp())
""")


Teraz wstawimy dane. Zauważ, że w zapytaniu `INSERT` pomijamy kolumny `order_sk` oraz `order_date`.
- `order_sk`: zostanie wygenerowane automatycznie (unikalny numer).
- `order_date`: zostanie wyliczone na podstawie `order_timestamp`.


## Sekcja 1.5: Managed vs External Tables (Demo)

**Cel**: Pokazanie różnicy w zachowaniu `DROP TABLE` dla tabel zarządzanych (Managed) i zewnętrznych (External).

- **Managed Table**: Databricks zarządza metadanymi ORAZ plikami. `DROP TABLE` usuwa wszystko.
- **External Table**: Databricks zarządza tylko metadanymi. `DROP TABLE` usuwa wpis w katalogu, ale pliki zostają.

> **Tip**: Nie potrzebujesz `EXTERNAL LOCATION` do tego demo! Możesz użyć ścieżki w **Unity Catalog Volume** lub nawet DBFS, aby zasymulować "zewnętrzną" lokalizację.

### 1.5.1 Managed Table - Pełne usunięcie
Tworzymy tabelę bez podawania `LOCATION`. Pliki trafią do domyślnej lokalizacji schematu (Managed Storage).

In [ ]:
# 1. Tworzymy Managed Table
spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{BRONZE_SCHEMA}.demo_managed")

spark.sql(f"""
    CREATE TABLE {CATALOG}.{BRONZE_SCHEMA}.demo_managed
    AS SELECT 1 as id, 'Managed Data' as info
""")

# Pobieramy lokalizację plików
managed_path = spark.sql(f"DESCRIBE DETAIL {CATALOG}.{BRONZE_SCHEMA}.demo_managed").select("location").first()[0]
print(f"Managed Table Path: {managed_path}")

# Sprawdzamy czy pliki istnieją
print(f"Files exist before DROP: {len(dbutils.fs.ls(managed_path)) > 0}")

# Usuwamy tabelę
spark.sql(f"DROP TABLE {CATALOG}.{BRONZE_SCHEMA}.demo_managed")

# Sprawdzamy czy pliki nadal istnieją (powinno rzucić błąd lub zwrócić False)
try:
    dbutils.fs.ls(managed_path)
    print("Files exist after DROP: TRUE (Unexpected!)")
except:
    print("Files exist after DROP: FALSE (Expected - Managed Table removed files)")

### 1.5.2 External Table (Unmanaged) - Zachowanie plików
Tworzymy tabelę podając `LOCATION`. Użyjemy ścieżki w Volume (lub tymczasowej), aby symulować zewnętrzny storage.

In [ ]:
external_path = f"{DATASET_BASE_PATH}/{raw_user}/demo_external_table"
dbutils.fs.rm(external_path, True) # Reset environment

# 1. Create External Table
spark.sql(f"""
    CREATE TABLE {CATALOG}.{BRONZE_SCHEMA}.demo_external
    LOCATION '{external_path}'
    AS SELECT 1 as id, 'External Data' as info
""")

# 2. Drop Table (Metadata only)
spark.sql(f"DROP TABLE {CATALOG}.{BRONZE_SCHEMA}.demo_external")

# 3. Verify Files Persist
files_exist = len(dbutils.fs.ls(external_path)) > 0
print(f"External Path: {external_path}")
print(f"Files persist after DROP: {'✅ YES' if files_exist else '❌ NO'}")

### Przykład 1.3: Modern Modeling - Identity & Generated Columns

**Cel:** Wykorzystanie nowoczesnych funkcji Delta Lake do automatyzacji modelu danych.

**Funkcje:**
- **Identity Columns**: Automatyczne generowanie unikalnych kluczy (Surrogate Keys).
- **Generated Columns**: Automatyczne wyliczanie wartości kolumn na podstawie innych (np. data z timestamp).


## Sekcja 2: Schema Evolution

**Wprowadzenie teoretyczne:**

Schema Evolution pozwala na kontrolowane dodawanie nowych kolumn do istniejących tabel Delta bez przerywania działania aplikacji. Delta Lake wspiera additive schema changes automatycznie.

### Przykład 2.1: Automatyczne dodawanie kolumn

**Cel:** Demonstracja automatycznej ewolucji schematu przy dodawaniu nowych kolumn

In [0]:
# Dane z dodatkową kolumną (customer_tier)
extended_customers = spark.createDataFrame([
    ("CUST010001", "New", "Customer", "new@example.com", "+48 111 222 333", "Warsaw", "MZ", "Poland", "2023-12-01", "Basic", "Premium"),
    ("CUST010002", "Another", "Customer", "another@example.com", "+48 444 555 666", "Krakow", "MP", "Poland", "2023-12-02", "Premium", "Standard")
], ["customer_id", "first_name", "last_name", "email", "phone", "city", "state", "country", "registration_date", "customer_segment", "customer_tier"])

**Włącz automatic schema evolution:**

In [0]:
%skip
from pyspark.sql.functions import col
from pyspark.sql.types import DateType

extended_customers = extended_customers.withColumn("registration_date", col("registration_date").cast(DateType()))

In [0]:
# Włącz automatic schema evolution
extended_customers.write \
    .format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta")

In [0]:
# Sprawdź nowy schemat
spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta").printSchema()

In [0]:
# Weryfikuj dane - nowa kolumna ma NULL dla starych rekordów
display(
    spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta")
    .select("customer_id", "first_name", "last_name", "customer_tier")
    .orderBy("customer_id")
)

In [0]:
# Dodaj CHECK constraint: customer_id musi zaczynać się od CUST
try:
    spark.sql(f"""
        ALTER TABLE {CATALOG}.{BRONZE_SCHEMA}.customers_delta
        ADD CONSTRAINT valid_customer_id CHECK (customer_id LIKE 'CUST%')
    """)
    print("Constraint 'valid_customer_id' dodany pomyślnie.")
except Exception as e:
    print(f"Informacja: {e}")

In [0]:
# Próba wstawienia błędnych danych (customer_id nie zaczyna się od CUST)
try:
    spark.sql(f"""
        INSERT INTO {CATALOG}.{BRONZE_SCHEMA}.customers_delta (customer_id, first_name, last_name, email, phone, city, state, country, registration_date, customer_segment)
        VALUES ('INVALID123', 'Bad', 'Customer', 'bad@example.com', '+48 000 000 000', 'Test', 'TS', 'Poland', '2023-01-01', 'Basic')
    """)
except Exception as e:
    print(f"Oczekiwany błąd Data Quality:\n{str(e)[:300]}...")

Constraint został dodany. Teraz spróbujmy wstawić dane, które go naruszają (customer_id nie zaczyna się od 'CUST'). 
Oczekujemy, że Delta Lake zablokuje tę operację i zwróci błąd `CheckConstraintViolation`.

## Sekcja 2.5: Data Quality & Constraints

**Wprowadzenie teoretyczne:**

Delta Lake pozwala na definiowanie **Constraints** (ograniczeń), które gwarantują jakość danych na poziomie tabeli. Działa to podobnie jak w tradycyjnych bazach danych SQL.

**Typy Constraints:**
- `NOT NULL`: Wymusza obecność wartości.
- `CHECK`: Wymusza spełnienie dowolnego warunku logicznego (np. `age > 0`).


## Sekcja 3: Time Travel i Disaster Recovery

**Wprowadzenie teoretyczne:**

Time Travel to kluczowa funkcjonalność Delta Lake umożliwiająca dostęp do poprzednich wersji danych. Bazuje na Copy-on-Write mechanizmie - każda zmiana tworzy nową wersję plików, a stare wersje pozostają dostępne.

**Disaster Recovery:**
Dzięki Time Travel możemy nie tylko czytać stare dane, ale także **przywracać** tabelę do poprzedniego stanu za pomocą polecenia `RESTORE`. To kluczowe w przypadku przypadkowego usunięcia danych lub błędnych aktualizacji.

### Przykład 3.1: Eksploracja historii tabeli

**Cel:** Użycie DESCRIBE HISTORY do analizy wszystkich operacji na tabeli

In [0]:
# Pokaż historię wszystkich operacji na tabeli
display(
    spark.sql(f"DESCRIBE HISTORY {CATALOG}.{BRONZE_SCHEMA}.customers_delta")
)

### Przykład 3.2: Time Travel queries

**Cel:** Dostęp do poprzednich wersji danych używając VERSION AS OF i TIMESTAMP AS OF

In [0]:
# Dostęp do danych z wersji 0 (przed schema evolution)
version_0_data = spark.sql(f"""
    SELECT *
    FROM {CATALOG}.{BRONZE_SCHEMA}.customers_delta VERSION AS OF 1
    ORDER BY customer_id
""")

In [0]:
display(version_0_data)

In [0]:
# Porównaj liczbę rekordów między wersjami
current_count = spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta").count()
version_0_count = spark.sql(f"SELECT * FROM {CATALOG}.{BRONZE_SCHEMA}.customers_delta VERSION AS OF 0").count()

display(
    spark.createDataFrame([
        ("Current version", current_count),
        ("Version 0", version_0_count)
    ], ["version", "record_count"])
)

In [0]:
# 1. Symulacja błędu: Przypadkowe usunięcie wszystkich danych
spark.sql(f"DELETE FROM {CATALOG}.{BRONZE_SCHEMA}.customers_delta")


In [0]:
print("Liczba rekordów po RESTORE:", spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta").count())


Tabela została przywrócona. Zweryfikujmy liczbę rekordów.


In [0]:
# 2. Naprawa: RESTORE do wersji sprzed usunięcia
# Pobieramy ostatnią dobrą wersję (przed DELETE)
last_good_version = spark.sql(f"DESCRIBE HISTORY {CATALOG}.{BRONZE_SCHEMA}.customers_delta").select("version").limit(2).collect()[1][0]

print(f"Przywracanie do wersji: {last_good_version}")

spark.sql(f"RESTORE TABLE {CATALOG}.{BRONZE_SCHEMA}.customers_delta TO VERSION AS OF {last_good_version}")


Teraz użyjemy **Time Travel**, aby znaleźć ostatnią poprawną wersję (sprzed usunięcia) i przywrócić tabelę poleceniem `RESTORE`.


In [0]:
print("Liczba rekordów po awarii:", spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta").count())


Ups! Usunęliśmy wszystkie dane. Sprawdźmy, czy tabela jest faktycznie pusta.


### Przykład 3.3: Disaster Recovery - RESTORE TABLE

**Cel:** Przywrócenie tabeli do stanu sprzed błędnej operacji (symulacja awarii).


## Sekcja 4: CRUD Operations

**Wprowadzenie teoretyczne:**

Delta Lake wspiera pełen zakres operacji CRUD (Create, Read, Update, Delete), co czyni go idealnym dla transakcyjnych workloadów w Data Lake. Wszystkie operacje są atomowe i ACID-compliant.

### Przykład 4.1: INSERT operation

**Cel:** Dodawanie nowych rekordów do istniejącej tabeli

In [0]:
# INSERT nowych klientów
spark.sql(f"""
    INSERT INTO {CATALOG}.{BRONZE_SCHEMA}.customers_delta
    (customer_id, first_name, last_name, email, phone, city, state, country, registration_date, customer_segment, customer_tier)
    VALUES 
        ('CUST020001', 'Insert', 'Customer1', 'insert1@example.com', '+48 111 111 111', 'Warsaw', 'MZ', 'Poland', '2023-12-10', 'Premium', 'Gold'),
        ('CUST020002', 'Insert', 'Customer2', 'insert2@example.com', '+48 222 222 222', 'Gdansk', 'PM', 'Poland', '2023-12-11', 'Basic', 'Silver')
""")

**Weryfikuj wstawienie:**

In [0]:
# Weryfikuj wstawienie
display(
    spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta")
    .filter(F.col("customer_id").like("CUST02%"))
    .orderBy("customer_id")
)

### Przykład 4.2: UPDATE operation

**Cel:** Aktualizacja istniejących rekordów w tabeli

In [0]:
# UPDATE customer tier dla specific customers
spark.sql(f"""
    UPDATE {CATALOG}.{BRONZE_SCHEMA}.customers_delta
    SET customer_tier = 'Platinum'
    WHERE customer_id IN ('CUST010001', 'CUST020001')
""")

**Weryfikuj aktualizację:**

In [0]:
# Weryfikuj aktualizację
display(
    spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta")
    .filter(F.col("customer_tier") == "Platinum")
)

### Przykład 4.3: DELETE operation

**Cel:** Usuwanie rekordów z tabeli Delta

In [0]:
# DELETE specific customer
spark.sql(f"""
    DELETE FROM {CATALOG}.{BRONZE_SCHEMA}.customers_delta
    WHERE customer_id = 'CUST020002'
""")

**Weryfikuj usunięcie:**

In [0]:
# Weryfikuj usunięcie
deleted_check = spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta") \
    .filter(F.col("customer_id") == "CUST020002") \
    .count()

display(
    spark.createDataFrame([
        ("Records with customer_id CUST020002", deleted_check)
    ], ["description", "count"])
)

## Sekcja 5: MERGE INTO Operations

**Wprowadzenie teoretyczne:**

MERGE INTO to potężna operacja umożliwiająca upsert (update + insert) w jednej transakcji. Szczególnie przydatna przy przetwarzaniu zmian z systemów transakcyjnych (CDC - Change Data Capture).

### Przykład 5.1: Podstawowy MERGE INTO

**Cel:** Demonstracja upsert operation - update istniejących i insert nowych rekordów

In [0]:
# Przygotuj dane do merge (mix updates i nowych rekordów)
merge_data = spark.createDataFrame([
    ("CUST010001", "Updated", "Name", "updated@example.com", "+48 999 999 999", "Poznan", "WP", "Poland", "2023-12-01", "VIP", "Diamond"),  # Update
    ("CUST030001", "Brand", "New", "brand.new@example.com", "+48 777 777 777", "Wroclaw", "DS", "Poland", "2023-12-15", "Basic", "Bronze"),   # Insert
    ("CUST030002", "Another", "New", "another.new@example.com", "+48 888 888 888", "Lodz", "LD", "Poland", "2023-12-16", "Premium", "Silver") # Insert
], ["customer_id", "first_name", "last_name", "email", "phone", "city", "state", "country", "registration_date", "customer_segment", "customer_tier"])

**Utwórz temporary view dla merge operation:**

In [0]:
# Utwórz temporary view dla merge operation
merge_data.createOrReplaceTempView("customer_updates")

**Wykonaj operację MERGE (Upsert):**

In [0]:
# MERGE INTO operation
spark.sql(f"""
    MERGE INTO {CATALOG}.{BRONZE_SCHEMA}.customers_delta AS target
    USING customer_updates AS source
    ON target.customer_id = source.customer_id
    
    WHEN MATCHED THEN
        UPDATE SET
            first_name = source.first_name,
            last_name = source.last_name,
            email = source.email,
            phone = source.phone,
            city = source.city,
            state = source.state,
            country = source.country,
            customer_segment = source.customer_segment,
            customer_tier = source.customer_tier
    
    WHEN NOT MATCHED THEN
        INSERT (customer_id, first_name, last_name, email, phone, city, state, country, registration_date, customer_segment, customer_tier)
        VALUES (source.customer_id, source.first_name, source.last_name, source.email, source.phone, source.city, source.state, source.country, source.registration_date, source.customer_segment, source.customer_tier)
""")

**Zweryfikuj wyniki MERGE:**

In [0]:
# Weryfikuj wyniki MERGE
display(
    spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta")
    .filter(F.col("customer_id").isin(["CUST010001", "CUST030001", "CUST030002"]))
    .orderBy("customer_id")
)

## Sekcja 6: Metadane i Analytics

**Wprowadzenie teoretyczne:**

Delta Lake oferuje bogate metadane o tabelach i operacjach. DESCRIBE DETAIL dostarcza informacji o strukturze plików, partitioning, i właściwościach tabeli.

### Przykład 6.1: DESCRIBE DETAIL

**Cel:** Analiza metadanych tabeli Delta

In [0]:
# Szczegółowe informacje o tabeli
display(
    spark.sql(f"DESCRIBE DETAIL {CATALOG}.{BRONZE_SCHEMA}.customers_delta")
)

### Przykład 6.2: Analiza historii operacji

**Cel:** Głębsza analiza historii i metryk operacji

In [0]:
# Historia z dodatkowymi metrykami
history_df = spark.sql(f"DESCRIBE HISTORY {CATALOG}.{BRONZE_SCHEMA}.customers_delta")

display(
    history_df.select(
        "version", 
        "timestamp", 
        "operation", 
        "operationMetrics.numTargetRowsInserted",
        "operationMetrics.numTargetRowsUpdated",
        "operationMetrics.numTargetRowsDeleted"
    )
)

### Przykład 6.3: Delta Log Internals (Deep Dive)

**Cel:** Zrozumienie jak Delta Lake zapewnia ACID, zaglądając "pod maskę" do plików JSON w `_delta_log`.


**Pobierz ścieżkę do tabeli i _delta_log:**

In [0]:
# Pobierz ścieżkę do tabeli
table_path = spark.sql(f"DESCRIBE DETAIL {CATALOG}.{BRONZE_SCHEMA}.customers_delta")
delta_log_path = f"{table_path}"

In [0]:
display(table_path)

**Wyświetl pliki w _delta_log:**

Poniżej podgląd zawartości ostatniego pliku transakcyjnego JSON. 
Zawiera on metadane o operacjach, takich jak:
- `add`: dodanie nowego pliku Parquet z danymi.
- `remove`: logiczne usunięcie pliku (np. przy operacji DELETE lub OPTIMIZE).
- `commitInfo`: metadane o samej transakcji (kto, kiedy, jaka operacja).


In [0]:
# View Delta table history using SQL
display(spark.sql(f"DESCRIBE HISTORY {CATALOG}.{BRONZE_SCHEMA}.customers_delta"))

**Analiza ostatniego pliku transakcji (JSON):**

## Sekcja 7: Optymalizacja (Wstęp)

**Wprowadzenie teoretyczne:**

Delta Lake oferuje szereg mechanizmów optymalizacyjnych. W tym notebooku skupimy się na podstawowej operacji **OPTIMIZE** (kompaktowanie plików) oraz **VACUUM** (czyszczenie).

> **Deep Dive:** Zaawansowane techniki takie jak **ZORDER BY**, **Partycjonowanie** oraz **Liquid Clustering** są szczegółowo omówione w notebooku **05_optimization_best_practices.ipynb**.


### Przykład 7.1: OPTIMIZE (File Compaction)

**Cel:** Kompaktowanie małych plików (small files problem) w większe, co poprawia wydajność odczytu.
Możemy opcjonalnie dodać klauzulę `ZORDER BY` (omówioną w notebooku 05), aby dodatkowo posortować dane.


**Wykonaj kompaktowanie plików:**

In [0]:
# OPTIMIZE tabeli
optimize_result = spark.sql(f"""
    OPTIMIZE {CATALOG}.{BRONZE_SCHEMA}.customers_delta
""")

display(optimize_result)

### Przykład 7.3: Liquid Clustering (Wzmianka)

**Nowoczesna alternatywa:**
Databricks wprowadził **Liquid Clustering** - nową technikę, która zastępuje tradycyjne partycjonowanie i ZORDER. 
Liquid Clustering automatycznie zarządza układem danych, dostosowując się do wzorców zapytań.

> **Deep Dive:** Szczegółowe omówienie i przykłady Liquid Clustering znajdują się w notebooku **05_optimization_best_practices.ipynb**.


### Przykład 7.2: VACUUM operation

**Cel:** Usunięcie starych plików (starszych niż retention period), które nie są już potrzebne do Time Travel.


**Wyłącz sprawdzenie retencji (tylko dla demo):**

In [0]:
# VACUUM - usuń pliki starsze niż 0 godzin (tylko dla demo!)
# W produkcji: domyślnie 7 dni, minimum 0 godzin z flagą
spark.sql("SET spark.databricks.delta.retentionDurationCheck.enabled = false")

**Uruchom VACUUM (usuń stare pliki):**

In [0]:
vacuum_result = spark.sql(f"""
    VACUUM {CATALOG}.{BRONZE_SCHEMA}.customers_delta RETAIN 0 HOURS
""")

display(vacuum_result)

## Sekcja 8: Change Data Feed

**Wprowadzenie teoretyczne:**

Change Data Feed (CDF) to feature Delta Lake umożliwiający tracking wszystkich zmian w tabeli. Każda operacja INSERT, UPDATE, DELETE jest rejestrowana z dodatkowymi metadanymi.

### Przykład 8.1: Włączenie Change Data Feed

**Cel:** Aktywacja CDF dla istniejącej tabeli

**Włącz Change Data Feed:**

In [0]:
# Włącz Change Data Feed
spark.sql(f"""
    ALTER TABLE {CATALOG}.{BRONZE_SCHEMA}.customers_delta 
    SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
""")

### Przykład 8.2: Generowanie zmian dla CDF

**Cel:** Wykonanie operacji które będą śledzzone przez CDF

**Wstaw nowy rekord (INSERT):**

In [0]:
# Wykonaj więcej zmian po włączeniu CDF
spark.sql(f"""
    INSERT INTO {CATALOG}.{BRONZE_SCHEMA}.customers_delta
    (customer_id, first_name, last_name, email, phone, city, state, country, registration_date, customer_segment, customer_tier)
    VALUES ('CUST040001', 'CDF', 'TestCustomer', 'cdf@example.com', '+48 555 555 555', 'Szczecin', 'ZP', 'Poland', '2023-12-20', 'Basic', 'Bronze')
""")

**Zaktualizuj rekord (UPDATE):**

In [0]:
spark.sql(f"""
    UPDATE {CATALOG}.{BRONZE_SCHEMA}.customers_delta
    SET customer_tier = 'Gold'
    WHERE customer_id = 'CUST040001'
""")

### Przykład 8.3: Odczyt Change Data Feed

**Cel:** Analiza wszystkich zmian zarejestrowanych przez CDF

In [0]:
# Sprawdź CDF properties
table_properties = spark.sql(f"SHOW TBLPROPERTIES {CATALOG}.{BRONZE_SCHEMA}.customers_delta")
cdf_enabled = table_properties.filter(F.col("key") == "delta.enableChangeDataFeed").count() > 0

display(
    spark.createDataFrame([
        ("Change Data Feed enabled", cdf_enabled)
    ], ["property", "status"])
)

**Odczytaj zmiany (CDF):**

In [0]:
# Batch read change data feed od określonej wersji
# Najpierw sprawdźmy aktualną wersję
current_version = spark.sql(f"DESCRIBE HISTORY {CATALOG}.{BRONZE_SCHEMA}.customers_delta").select("version").first()[0]
cdf_start_version = max(0, current_version - 3)  # Ostatnie 3 wersje

display(current_version)

In [0]:
changes_batch = spark.read \
    .format("delta") \
    .option("readChangeFeed", "true") \
    .option("startingVersion", cdf_start_version) \
    .table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta")

    

In [0]:
display(changes_batch)

In [0]:
display(
    changes_batch.select(
        "customer_id", "first_name", "last_name", "customer_tier", 
        "_change_type", "_commit_version", "_commit_timestamp"
    )
)

## Sekcja 9: Zero-Copy Cloning

**Wprowadzenie teoretyczne:**

Delta Lake umożliwia tworzenie kopii tabel (Clones).
- **SHALLOW CLONE**: Kopiuje tylko metadane (Delta Log), a dane fizyczne pozostają te same. Idealne do testów, QA, eksperymentów. Koszt storage = prawie zero.
- **DEEP CLONE**: Kopiuje metadane i dane fizyczne. Idealne do archiwizacji lub migracji.


**Utwórz SHALLOW CLONE:**

In [0]:
%skip
spark.sql(f"""DROP TABLE IF EXISTS {CATALOG}.{BRONZE_SCHEMA}.customers_test_clone""")

In [0]:
# Utwórz SHALLOW CLONE do testów
spark.sql(f"""
CREATE TABLE {CATALOG}.{BRONZE_SCHEMA}.customers_test_clone
SHALLOW CLONE {CATALOG}.{BRONZE_SCHEMA}.customers_delta
""")


**Wykonaj operację DELETE na klonie:**

In [0]:
# Wykonaj destrukcyjną operację na klonie (nie wpływa na oryginał!)
spark.sql(f"DELETE FROM {CATALOG}.{BRONZE_SCHEMA}.customers_test_clone WHERE customer_tier = 'Premium'")

**Porównaj liczniki (izolacja):**

In [0]:
# Porównaj liczniki
orig_count = spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta").count()
clone_count = spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_test_clone").count()

print(f"Oryginał: {orig_count}")
print(f"Klon (po delete): {clone_count}")


Porównanie liczby rekordów potwierdza izolację. Oryginał powinien mieć więcej rekordów niż klon po usunięciu.


In [0]:
display(delta_table)

**SQL Equivalent:**

In [0]:
# SQL approach
spark.sql(f"""
    MERGE INTO {CATALOG}.{BRONZE_SCHEMA}.customers_delta AS target
    USING (SELECT 'CUST050002' as customer_id, 'SQL' as first_name, 'Customer' as last_name,
                  'sql@example.com' as email, '+48 333 333 333' as phone,
                  'Lublin' as city, 'LB' as state, 'Poland' as country,
                  '2023-12-26' as registration_date, 'Basic' as customer_segment,
                  'Gold' as customer_tier) AS source
    ON target.customer_id = source.customer_id
    WHEN MATCHED THEN UPDATE SET *
    WHEN NOT MATCHED THEN INSERT *
""")

In [0]:
display(spark.sql(f"SELECT * FROM {CATALOG}.{BRONZE_SCHEMA}.customers_delta WHERE customer_id = 'CUST050003'"))

**Porównanie:**
- **Wydajność**: Identyczna - obydwa używają Catalyst optimizer
- **Kiedy używać PySpark**: Programatic ETL, complex business logic, integration z ML pipelines
- **Kiedy używać SQL**: Ad-hoc analysis, reporting, BI tools integration, łatwiejsze dla analityków

**Weryfikacja podstawowych metryk:**

In [0]:
# Weryfikacja wyników
final_count = spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta").count()
final_schema_cols = len(spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.customers_delta").columns)
history_count = spark.sql(f"DESCRIBE HISTORY {CATALOG}.{BRONZE_SCHEMA}.customers_delta").count()

print(
    ("Total records", final_count),
    ("Schema columns", final_schema_cols),
    ("History versions", history_count)
)

## Walidacja i weryfikacja

### Checklist - Co powinieneś uzyskać:
- [ ] Tabela Delta utworzona z automatycznym schema enforcement
- [ ] Schema evolution - dodana kolumna customer_tier
- [ ] Time Travel queries działają dla poprzednich wersji
- [ ] CRUD operations (INSERT, UPDATE, DELETE) wykonane poprawnie
- [ ] MERGE INTO zaimplementowane z upsert logic
- [ ] Optymalizacja OPTIMIZE i ZORDER zastosowana
- [ ] Change Data Feed włączony i rejestruje zmiany

### Komendy weryfikacyjne:

## Best Practices

### Wydajność:
- Używaj ZORDER BY dla kolumn często występujących w WHERE clauses
- Regularnie uruchamiaj OPTIMIZE dla kompaktowania small files
- Partitioning tylko dla bardzo dużych tabel (TB+) ze skewed data

### Jakość kodu:
- Zawsze używaj explicit schema zamiast inferSchema w production
- Implementuj schema evolution strategy dla backward compatibility
- Używaj MERGE INTO zamiast separate DELETE + INSERT operations

### Data Quality:
- Włącz Change Data Feed dla audit trails i compliance
- Regularne backup przez Time Travel snapshots
- Implement data validation rules w Delta constraints

### Governance:
- Ustaw odpowiednie retention periods dla compliance requirements
- Używaj Unity Catalog permissions dla row/column level security
- Dokumentuj schema changes i business logic w table comments

## Troubleshooting

### Problem 1: Schema enforcement błąd
**Objawy:**
- AnalysisException przy INSERT/MERGE z incompatible schema
- "Cannot write incompatible datatype" message

**Rozwiązanie:**
```python
# Użyj mergeSchema option dla schema evolution
df.write.format("delta").option("mergeSchema", "true").mode("append")
```

### Problem 2: Time Travel - version not found
**Objawy:** 
File not found dla określonej wersji po VACUUM

**Rozwiązanie:** 
Sprawdź retention period i dostępne wersje przez DESCRIBE HISTORY

### Problem 3: VACUUM usuwa zbyt dużo plików
**Objawy:** Time Travel queries failują po VACUUM

**Rozwiązanie:** 
Ustaw odpowiedni retention period (domyślnie 7 dni minimum)

### Debugging tips:
- Użyj `DESCRIBE HISTORY` aby zrozumieć operacje na tabeli
- Sprawdź `DESCRIBE DETAIL` dla metadanych o plikach
- Weryfikuj table properties przez `SHOW TBLPROPERTIES`
- Monitoruj `_delta_log/` folder dla troubleshooting

## Podsumowanie

### Co zostało osiągnięte:
- Demonstracja Delta Lake ACID properties i schema enforcement
- Hands-on Schema Evolution z automatic column addition
- Time Travel queries dla historical data access
- Kompletne CRUD operations (CREATE, READ, UPDATE, DELETE)
- Advanced MERGE INTO dla upsert scenarios
- Performance optimization z OPTIMIZE, ZORDER, VACUUM
- Change Data Feed dla comprehensive audit trails

### Kluczowe wnioski:
1. **Delta Lake = Data Lake + ACID**: Łączy elastyczność Data Lake z niezawodnością transakcyjną
2. **Schema Evolution bezpiecznie**: Additive changes są automatyczne, breaking changes wymagają planowania
3. **Time Travel + Copy-on-Write**: Każda wersja jest preserved, umożliwiając rollback i audit

### Quick Reference - Najważniejsze komendy:

| Operacja | PySpark | SQL |
|----------|---------|-----|
| Create Delta Table | `df.write.format("delta").saveAsTable()` | `CREATE TABLE USING DELTA` |
| Time Travel | `spark.read.format("delta").option("versionAsOf", 1)` | `SELECT * FROM table VERSION AS OF 1` |
| MERGE | `DeltaTable.forName().merge().execute()` | `MERGE INTO target USING source` |
| Optimize | N/A | `OPTIMIZE table ZORDER BY col` |
| History | N/A | `DESCRIBE HISTORY table` |

### Następne kroki:
- **Kolejny notebook**: 02_medallion_architecture.ipynb
- **Warsztat praktyczny**: 01_delta_medallion_workshop.ipynb
- **Materiały dodatkowe**: Delta Lake documentation, best practices guides

## Czyszczenie zasobów

Posprzątaj zasoby utworzone podczas notebooka:

In [0]:
# Opcjonalne czyszczenie zasobów testowych
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone dane

# spark.sql(f"DROP TABLE IF EXISTS {CATALOG}.{BRONZE_SCHEMA}.customers_delta")
# spark.sql("DROP VIEW IF EXISTS customer_updates")
# spark.catalog.clearCache()

# display(spark.createDataFrame([("Zasoby zostały wyczyszczone", "✓")], ["status", "result"]))